**Exercise:** In preparation for an alien invasion, the Earth Defense League (EDL) has been working on new missiles to shoot down space invaders.  Of course, some missile designs are better than others; let's assume that each design has some probability of hitting an alien ship, `x`.

Based on previous tests, the distribution of `x` in the population of designs is approximately uniform between 0.1 and 0.4.

Now suppose the new ultra-secret Alien Blaster 9000 is being tested.  In a press conference, an EDL general reports that the new design has been tested twice, taking two shots during each test.  The results of the test are confidential, so the general won't say how many targets were hit, but they report: "The same number of targets were hit in the two tests, so we have reason to think this new design is consistent."

Is this data good or bad; that is, does it increase or decrease your estimate of `x` for the Alien Blaster 9000?

Hint: If the probability of hitting each target is $x$, the probability of hitting one target in both tests
is $\left[2x(1-x)\right]^2$.

## Hitting two spacecrafts in four shots
Note that mathematically there's no difference between shooting four times and making two experiments of two shots or four experiments of one shot


| 1<sup>st</sup> shot | 2<sup>nd</sup> shot | 3<sup>rd</sup> shot | 4<sup>th</sup> shot |
| ------------------- | ------------------- | ------------------- | ------------------- |
| hs | (1-hs) | hs | (1-hs) |
| hs | (1-hs) | (1-hs) | hs |
| (1-hs) | hs | hs | (1-hs) |
| (1-hs) | hs | (1-hs) | hs |

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# the hypothesis space represents the probability of hitting an alien spacecraft
hs = np.linspace(.1, .4, 101)

# we have a uniform distribution of weapons
prior = np.full(101, 1/101)

# get the likelihoods for hitting no one, one and both spacecrafts in two tests
# for no hit at all it's just the joint probability of failing four times:
# (1-hs)^4 (as hs is the probability of hitting a space craft) 
like0 = (1 - hs)**4

# likewise for hitting both spacecrafts in both tests, it's the joint
# probability of hitting four times. 
like2 = hs**4

# Finally, hitting just one in both tests is slightly trickier. There are four
# ways to achieve this outcome (see the table above)
like1 = 4*(hs * (1 - hs))**2  

likes = (like0, like1, like2)

def get_confidence_interval(posterior):
    """Return the indices that represent the bounds for a 90% ci."""
    cdf = posterior.cumsum()
    lower = np.where(cdf >= .05)[0][0]
    upper = np.where(cdf <= .95)[0][-1]
    return lower, upper

# Compute posteriors
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("Weapon performance")
ax.set_ylabel("Probability of the performance")

for n, like in enumerate(likes):
    posterior = prior * like
    posterior /= posterior.sum()
    sns.lineplot(x=hs, y=posterior, label=f"like{n}", ax=ax);
    lower, upper = get_confidence_interval(posterior)
    lower, upper = [round(hs[bound], 3) for bound in (lower, upper)]
    print(f"For like{n} the 90% of the designs live between {lower} and {upper}")
    width = round(100 * (upper - lower)/(.4-.1), 2)
    print(f"90% ci width: {width}%")

In above chart there's a big deal going on:

Like0 is telling us that if we had been told that the experiment had hit no spacecraft it would be reasonable to think that most of our weapons (90%) would live spread out accross the accuracy spectrum (10.6% and 36%) being more probable the lower values than the upper.
Like1 improves like0 outcomes (in the case we had been told that one target had been hit in each test) raising the lower bound of the location of the 90% of the weapons to the 14.5%. Finally like2 shows actually the most performance for 4 shots showing that the desings live in the narrowest area (.17 compared with .26 & .24 of like0 and like1) in the upper side of the accuracy spectrum.

### Mixing likelihoods
However, above approach doesn't answer the question *is this data good or bad; that is, does it increase or decrease your estimate of `x` for the Alien Blaster 9000?* as we weren't told how many targets were hit, only that the same amount, so we should mix likelihoods since anything could have happened

In [ ]:
# Mix the likelihoods to see who has more weight
likelihood = np.sum(likes, axis=0)

posterior = prior * likelihood
posterior /= posterior.sum()

lower, upper = get_confidence_interval(posterior)
lower, upper = [round(hs[bound], 3) for bound in (lower, upper)]
print(f"For the mixed likelihood the 90% of the designs live between {lower} and {upper}")


fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("Weapon performance")
ax.set_ylabel("Probability of the performance")

sns.lineplot(x=hs, y=posterior, ax=ax);

This shows us that **with the information we were given**, namely that the same number of targets were hit, it's more reasonable to think that:
* our weapons live very spread out in the accuracy spectrum, and 
* although like1 and like2 are pulling up the right hand side, this pull is not powerful enough to give more weight to the high performant weapons, so low performant ones are more probable.